# Introduction to Algorithms, 4th Edition: Part III Data Structures - Chapter 10 Elementary Data Structures - Section 11.3 Hash Functions

For hashing to work well, it needs a good hash function. Along with being efficiently computable, what properties does a good hash function have? How do you design good hash functions?

This section first attempts to answer these questions based on two ad hoc approaches for creating hash functions: hashing by division and hashing by multiplication. Although these methods work well for some sets of input keys, they are limited because they try to provide a single fixed hash function that works well on any data - an approach called ***static hashing***.

We then see that provably good average-case performance for *any* data can be obtained by designing a suitable *family* of hash functions and choosing a hash function at random from this family at runtime, independent of the data being hashed. The approach we examine is called random hashing. A particular kind of random hashing, universal hashing, works well. As we saw with quicksort in Chapter 7, randomization is a powerful algorithmic design tool.

## What Makes a Good Hash Function?
A good hash function satisfies (approximately) the assumption of independent uniform hashing: each key is equally likely to hash to any of the $m$ slots, independently of where any other keys have hashed to. What does "equally likely" mean here? If the hash function is fixed, any probabilities would have to be based on the probability distribution of the input keys.

Unfortunately, you typically have no way to check this condition, unless you happen to know the probability distribution from which the keys are drawn. Moreover, the kys might not be drawn independently.

Occasionally you might know the distribution. For example, if you know that the keys are random real numbers $k$ independently and uniformly distributed in the range $0 \leq k < 1$, then the hash function
$$
h(k) = \lfloor km \rfloor
$$
satisfies the condition of independent uniform hashing.

A good static hashing approach derives a hash value in a way that you expect to be independent of any patterns that might exist in the data. For example, the "division method" (discussed in Section 11.3.1) computes the hash value as the remainder when the key is divided by a specific prime number. This method may give good results if you (somehow) choose a prime number that is unrelated to any patterns in the distribution of the keys.

Random hashing, described in Section 11.3.2, picks the hash function to be used at random from a suitable family of hash functions. This approach removes any need to know anything about the probability distribution of the input keys, as the randomization necessary for good average-case behavior then comes from the (known) random process used to pick the hash function from the family of hash functions, rather than from the (unknown) process used to create the input keys. We recommend that you use random hashing.

## Keys are Integers, Vectors, or Strings
In practice, a hash function is designed to handle keys that are one of the following two types:
* A short nonnegative integer that fits in a $w$-bit machine word. Typical values for $w$ would be $32$ or $64$.
* A short vector of nonnegative integers, each of bounded size. For example, each element might be an $8$-bit byte, in which case the vector is often called a (byte) string. The vector might be of variable length.

To begin, we assume that keys are short nonnegative integers. Handling vector keys is more complicated and discussed in Sections 11.3.5 and 11.5.2.

## 11.3.1 Static Hashing
Static hashing uses a single, fixed hash function. The only randomization available is through the (usually unknown) distribution of input keys. This section discusses two standard approaches for static hashing: the division method and the multiplication method. Although static hashing is no longer recommended, the multiplication method also provides a good foundation for "nonstatic" hashing - better known as random hashing - where the hash function is chosen at random from a suitable family of hash functions.

### The Division Method
The ***division method*** for creating hash functions maps a key $k$ into one of $m$ slots by taking the remainder of $k$ divided by $m$. That is, the hash function is
$$
h(k) = k\bmod{m}
$$
For example, if the hash table has size $m = 12$ and the key is $k = 100$, then $h(k) = 4$. Since it requires only a single division operation, hashing by division is quite fast.

The division method may work well when $m$ is a prime not too close to an exact power of $2$. There is no guarantee that this method provides good average-case performance, however, and it may complicate applications since it constrains the size of the hash table to be prime.

### The Multiplication Method
The general ***multiplication method*** for creating hash functions operates in two steps. First, multiply the key $k$ by a constant $A$ in the range $0 < A < 1$ and extract the fractional part of $kA$. Then, multiply this value by $m$ and take the floor of the result. That is, the hash function is
$$
h(k) = \lfloor m(kA\bmod{1}) \rfloor
$$
where "$kA\bmod{1}$" means the fractional part of $kA$, that is, $kA - \lfloor kA \rfloor$. The general multiplication method has the advantage that the value of $m$ is not critical and you can choose it independently of how you choose the multiplicative constant $A$.

### The Multiply-Shift Method
In practice, the multiplication method is best in the special case where the number $m$ of hash-table slots is an exact power of $2$, so that $m = 2^\ell$ for some integer $\ell$, where $\ell \leq w$ and $w$ is tbe number of bits in a machine word. If you choose a fixed $w$-bit positive integer $a = A2^w$, where $0 < A < 1$ as in the multiplication method so that $a$ is in the range $0 < a < 2^w$, you can implement the function on most computers as follows. We assume that a key $k$ fits into a single $w$-bit word.

Referring to Figure 11.4, first multiply $k$ by the $w$-bit integer $a$. The result is a $2w$-bit value $r_1 2^w + r_0$, where $r_1$ is the high-order $w$-bit word of the product and $r_0$ is the low-order $w$-bit word of the product. The desired $\ell$-bit hash value consist of the $\ell$ most significant bits of $r_0$. (Since $r_1$ is ignored, the hash function can be implemented on a computer that produces only a $w$-bit product given two $w$-bit inputs, that is, where the multiplication operation computes modulo $2^w$.)

In other words, you define the hash function $h = h_a$, where
$$
h_a(k) = ka\bmod{2^w} \ggg w - \ell \qquad \textbf{(11.2)}
$$
for a fixed nonzero $w$-bit value $a$. Since the product $ka$ of two $w$-bit words occupies $2w$ bits, taking this product modulo $2^w$ zeroes out the high-order $w$ bits (r_1), leaving only the low-order $w$ bits (r_0). The $ggg$ operator performs a logical right shift by $w - \ell$ bits, shifting zeros into the vacated positions on the left, so that the $\ell$ most significant bits of $r_0$ move into the $\ell$ rightmost positions. (It's the same as dividing by $2^{w - \ell}$ and taking the floor of the result.) The resulting value equals the $\ell$ most significant bits of $r_0$. The hash function $h_a$ can be implemented with three machine instructions: multiplication, subtraction, and logical right shift.

<img src="Figure 11.4.png" alt="Figure 11.4" width="750"/>

Even though the multiply-shift method is fast, it doesn't provide any guarantee of good average-case performance. The universal hashing approach presented in the next section provides such a guarantee. A simple randomized variant of the multiply-shift method works well on the average, when the program begins by picking $a$ as a randomly chosen odd integer.

## 11.3.2 Random Hashing
Suppose that a malicious adversary chooses the keys to be hashed by some fixed hash function. Then the adversary can choose $n$ keys that all hash into the same slot, yielding an average retrieval time of $\Theta(n)$. Any static hash function is vulnerable to such terrible worst-case behavior. The only effective way to improve the situation is to choose the hash function *randomly* in a way that is *independent* of the keys that are actually going to be stored. This approach is called ***random hashing***. A special case of this approach, called ***universal hashing***, can yield probably good performance on average when collisions are handled by chaining, no matter which keys the adversary chooses.

To use random hashing, at the beginning of program execution you select the hash function at random from a suitable family of functions. As in the case of quicksort, randomization guarantees that no single input always evokes worst-case behavior. Because you randomly select the hash function, the algorithm can behave differently on each execution, even for the same set of keys to be hashed, guaranteeing good average-case performance.

Let $\mathscr{H}$ be a finite family of hash functions that map a given universe $U$ of keys into the range $\{0, 1, \ldots, m - 1\}$. Such a family is said to be ***universal*** if for each pair of distinct keys $k_1, k_2 \in U$, the number of hash function $h \in \mathscr{H}$ for which $h(k_1) = h(k_2)$ is at most $\frac{\mathscr{H}}{m}$. In other words, with a hash function randomly chosen from $\mathscr{H}$, the chance of a collision between distinct keys $k_1$ and $k_2$ is no more than the chance $\frac{1}{m}$ of a collision if $h(k_1)$ and $h(k_2)$ were randomly and independently chosen from the set $\{0, 1, \ldots, m - 1\}$.

Independent uniform hashing is the same as picking a hash function uniformly at random from a family of $m^n$ hash functions, each member of that family mapping the $n$ keys to the $m$ hash values in a different way.

Every independent uniform random family of hash functions is universal, but the converse need not to be true: consider the case where $U = \{0, 1, \ldots, m - 1\}$ and the only hash function in the family is the identity function. The probability that two distinct keys collide is zero, even though each key hashes to a fixed value.

The following corollary to Theorem 11.2 says that universal hashing provides the desired payoff: it becomes impossible for an adversary to pick a sequence of operations that forces the worst-case running time.

### *Corollary 11.3*
Using universal hashing and collision resolution by chaining an initially empty table with $m$ slots, it takes $\Theta(s)$ expected time to handle any sequence of $s$ $\texttt{Insert}$, $\texttt{Search}$, and $\texttt{Delete}$ operations containing $n = O(m)$ $\texttt{Insert}$ operations.

***Proof*** The $\texttt{Insert}$ and $\texttt{Delete}$ operations take constant time. Since the number $n$ of insertions is $O(m)$, we have that $\alpha = O(1)$. Furthermore, the expected time for each $\texttt{Search}$ operation is $O(1)$, which can be seen by examining the proof of Theorem 11.2. That analysis depends only on collision probabilities, which are $\frac{1}{m}$ for any pair $k_1, k_2$ of keys by the choice of an independent uniform hash function in that theorem. Using a universal family of hash functions here instead of using independent uniform hashing changes the probability of collision from $\frac{1}{m}$ to at most $\frac{1}{n}$. By linearity of expectation, therefore, the expected time for the entire sequence of $s$ operations is $O(s)$. Since each operation takes $\Omega(1)$ time, the $\Theta(s)$ bound follows.

<div style="text-align: right"> $\blacksquare$ </div>

## 11.3.3 Achievable Properties of Random Hashing
There is a rich literature on the properties a family $\mathscr{H}$ of hash functions can have, and how they relate to the efficiency of hashing. We summarize a few of the most interesting ones here.

Let $\mathscr{H}$ be a family of hash functions, each with domain $U$ and range $\{0, 1, \ldots, m - 1\}$, and let $h$ be any hash function that is picked uniformly at random from $\mathscr{H}$. The probabilities mentioned are probabilities over the picks of $h$.

* The family $\mathscr{H}$ is ***uniform*** if for any key $k$ in $U$ and any slot $q$ in the range $\{0, 1, \ldots, m - 1\}$, the probability that $h(k) = q$ is $\frac{1}{m}$.
* The family $\mathscr{H}$ is ***universal*** if for any distinct keys $k_1$ and $k_2$ in $U$, the probability that $h(k_1) = h(k_2)$ is at most $\frac{1}{m}$.
* The family $\mathscr{H}$ of hash functions is ***$\epsilon$-universal*** if for any distinct keys $k_1$ and $k_2$ in $U$, the probability that $h(k_1) = h(k_2)$ is at most $\epsilon$. Therefore, a universal family of hash functions is also $\frac{1}{m}$-universal.
* The family $\mathscr{H}$ is ***d-dependent*** if for any distinct keys $k_1, k_2, \ldots, k_d$ in $U$ and any slots $q_1, q_2, \ldots, q_d$, not necessarily distinct, in $\{0, 1, \ldots, m - 1\}$ the probability that $h(k_i) = q_i$ for $i = 1, 2, \ldots, d$ is $\frac{1}{m^d}$

Universal hash-function families are of particular interest, as they are the simplest type supporting provably efficient hash-table operations for any input data set. Many other interesting and desirable properties, such as those noted above, are also possible and allow for efficient specialized hash-table operations.

## 11.3.4 Designing a Universal Family of Hash Functions
This section presents two ways to design a universal (or $\epsilon$-universal) family of hash functions: one based on number theory and another based on a randomized variant of the multiply-shift method presented in Section 11.3.1. The first method is a bit easier to prove universal, but the second method is newer and faster in practice.

### A Universal Family of Hash Functions based on Number Theory
We can design a universal family of hash functions using a little number theory. You may wish to refer to Chapter 31 if you are unfamiliar with basic concepts in number theory.

Begin by choosing a prime number $p$ large enough so that every possible key $k$ lies in the range $0$ to $p - 1$, inclusive. We assume here that $p$ has a "reasonable" length. (See Section 11.3.5 for a discussion of methods for handling long input keys, such as variable-length strings.) Let $\mathbb{Z}_p$ denote the set $\{0, 1, \ldots, p - 1\}$, and let $\mathbb{Z}^*_p$ denote the set $\{1, 2, \ldots, p - 1\}$. Since $p$ is prime, we can solve equations modulo $p$ with the methods given in Chapter 31. Because the size of the universe of keys is greater than the number of slots in the hash table (otherwise, just use direct addressing), we have $p > m$.

Given any $a \in \mathbb{Z}^*_p$ and any $b \in \mathbb{Z}_p$, define the hash function $h_{ab}$ as a linear transformation followed by reductions modulo $p$ and then modulo $m$:
$$
h_{ab}(k) = ((ak + b)\bmod{p})\bmod{m}. \qquad \textbf{(11.3)}
$$